# Gearshift test results

To facilitate the comparison of results between the MATLAB tool and the Python tool, we have created this notebook that reads the MATLAB and Python results and compares the output values.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# Libraries that going to be used
import pandas as pd
from os.path import abspath, join, dirname
from pandas.util import hash_pandas_object

pd.set_option('display.max_rows', 125)

The downscaled factor (f_dsc) value can not be compared between Python and MATLAB because the value f_dsc in MATLAB is the calculated downscaling factor (this value is only used when, in the input file, the calc_dsc value is equal to 1). In Python, it represents the downscaling factor, therefore the two values are not the same.

In [3]:
column_rename = {# 'case': 'Case', 
' r_max': 'Required To Rated Power Ratio', 
#' f_dsc': 'Downscaling Factor', 
'   v_sum': 'Total Checksum', 
' v_max': 'Max Vehicle Speed', 
'  d_cycle': 'Total Distance',
'  g_avg': 'Average Gear Output', 
'  n_max1': 'Adjusted Max95 Engine Speed', 
'  n_max2': 'Max Engine Speed Cycle Output', 
'  n_max3': 'Max Engine Speed Reachable Output', 
'   n_max': 'Max Engine Speed Output',
' v_max_c': 'Max Vehicle Speed Cycle Output', 
' v_max_v': 'Max Vehicle Speed Reachable Output', 
' g_v_max': 'Gear Max Vehicle Speed Reachable Output', 
'  n_min1': 'Min Drive Engine Speed1st Output', 
' n_min12': 'Min Drive Engine Speed1st To2nd Output', 
' n_min2d': 'Min Drive Engine Speed2nd Decel Output',
'  n_min2': 'Min Drive Engine Speed2nd Output', 
'  n_min3': 'Min Drive Engine Speed Greater2nd Output',
' v_x_g_sum': 'Checksum Vx Gear Output'
} 

In [4]:
def highlight_false(val):
    color = 'red' if val == False else 'green'
    return 'background-color: %s' % color

def highlight_big_diff(val):
    color = 'red' if val > 0.2 else 'green'
    return 'background-color: %s' % color

def create_df_comparison_from_dict_results(results_dict, bool_comparison=True): 
    
    list_dfs = list()
    for key in results_dict.keys(): 
        _df = pd.DataFrame.from_dict(results_dict[key])
        
        if bool_comparison: 
            _df[key] = _df['python'] == _df['matlab']
        else:
            _df[key] = abs(_df['python'] - _df['matlab'])
        
        list_dfs.append(_df[[key]])
    
    if bool_comparison:
        return pd.concat(list_dfs, axis=1).T.style.applymap(highlight_false)
    else: 
        return pd.concat(list_dfs, axis=1).T.style.applymap(highlight_big_diff)

## Read Output Data

In [5]:
python_path = r'C:\Users\andre\Documents\European Comision\WLTP\_output\20210216_150821-'
matlab_path = r'C:\Users\andre\Documents\European Comision\WLTP\matlab_test\test_'

In [6]:
%%time

results = dict()

for case_no in range(1, 126):
    
    # python data
    df_py = pd.ExcelFile(python_path + str(case_no) + '.xlsx')
    summary_py = pd.read_excel(df_py, 'Summary')
    original_trace_py = pd.read_excel(df_py, 'Original Trace')
    applicable_trace_py = pd.read_excel(df_py, 'Applicable Trace')
    time_series_py = pd.read_excel(df_py, 'Time Series')
    available_powers_output_py = pd.read_excel(df_py, 'Available Powers Output')
    required_engine_speeds_output_py = pd.read_excel(df_py, 'Required Engine Speeds Output')
    
    # matlab data
    original_trace_ml = pd.read_csv(matlab_path + str(case_no) + '\OriginalTrace.txt', sep=",", header=None)
    original_trace_ml.rename(columns = {0: 'Trace Times', 1: 'Vehicle Speeds'}, inplace=True)
    applicable_trace_ml = pd.read_csv(matlab_path + str(case_no) + '\ApplicableTrace.txt', sep=",", header=None)
    applicable_trace_ml.rename(columns = {0: 'Trace Times', 1: 'Vehicle Speeds'}, inplace=True)
    available_powers_output_ml = pd.read_csv(matlab_path + str(case_no) + '\AvailablePowersOutput.txt', 
                                             sep=",", header=None)
    available_powers_output_ml = available_powers_output_ml.fillna(0)
    required_engine_speeds_output_ml = pd.read_csv(matlab_path + str(case_no) + '\RequiredEngineSpeedsOutput.txt', 
                                                   sep=",", header=None)
    required_engine_speeds_output_ml = required_engine_speeds_output_ml.fillna(0)
    gears_output_ml = pd.read_csv(matlab_path + str(case_no) + '\GearsOutput.txt', 
                                  sep=",", header=None)
    gears_corrections_output_ml = pd.read_csv(matlab_path + str(case_no) + '\GearCorrectionsOutput.txt', 
                                              sep=",", header=None)
    required_vehicles_speed_output_ml = pd.read_csv(matlab_path + str(case_no) + '\RequiredVehicleSpeedsOutput.txt',
                                                    sep=",", header=None)
    required_powers_output_ml = pd.read_csv(matlab_path + str(case_no) + '\RequiredPowersOutput.txt', 
                                            sep=",", header=None)
    clutch_disengaged_output_ml = pd.read_csv(matlab_path + str(case_no) + '\ClutchDisengagedOutput.txt', 
                                              sep=",", header=None)
    clutch_undefined_output_ml = pd.read_csv(matlab_path + str(case_no) + '\ClutchUndefinedOutput.txt', 
                                             sep=",", header=None)
    clutch_HST_output_ml = pd.read_csv(matlab_path + str(case_no) + '\ClutchHSTOutput.txt', 
                                       sep=",", header=None)
    clutch_HST_output_ml = clutch_HST_output_ml.T
    summary_ml = pd.read_csv(matlab_path + str(case_no) + '\case_result.txt', sep=",",)
    
    # append results
    results[case_no] = dict()
    
    results[case_no]['python'] = dict()
    results[case_no]['python']['summary'] = summary_py[list(column_rename.values())]
    results[case_no]['python']['original_trace_times'] = original_trace_py['Trace Times']
    results[case_no]['python']['original_trace_vehicle_speeds'] = original_trace_py['Vehicle Speeds']
    results[case_no]['python']['applicable_trace_times'] = applicable_trace_py['Trace Times']
    results[case_no]['python']['applicable_trace_vehicle_speeds'] = applicable_trace_py['Vehicle Speeds']
    results[case_no]['python']['ts_trace_times'] = time_series_py['Trace Times Output']
    results[case_no]['python']['ts_gears_out'] = time_series_py['Gears Output']
    time_series_py["Gear Corrections Output"] = \
                time_series_py["Gear Corrections Output"].apply(lambda x: x.replace('  ', ' '))
    results[case_no]['python']['ts_gears_correction_out'] = time_series_py["Gear Corrections Output"]
    results[case_no]['python']['ts_required_vehicle_speeds_out'] = time_series_py['Required Vehicle Speeds Output']
    results[case_no]['python']['ts_required_powers_out'] = time_series_py['Required Powers Output']
    results[case_no]['python']['ts_clutch_disengaged_out'] = time_series_py['Clutch Disengaged Output']
    results[case_no]['python']['ts_clutch_undefined_out'] = time_series_py['Clutch Undefined Output']
    results[case_no]['python']['ts_clutch_hst_out'] = time_series_py["Clutch H S T Output"]
    
    results[case_no]['matlab'] = dict()
    results[case_no]['matlab']['summary'] = summary_ml
    results[case_no]['matlab']['summary'].rename(columns=column_rename, inplace=True)
    results[case_no]['matlab']['summary'] = results[case_no]['matlab']['summary'][list(column_rename.values())]
    results[case_no]['matlab']['original_trace_times'] = original_trace_ml['Trace Times']
    results[case_no]['matlab']['original_trace_vehicle_speeds'] = original_trace_ml['Vehicle Speeds']
    results[case_no]['matlab']['applicable_trace_times'] = applicable_trace_ml['Trace Times']
    results[case_no]['matlab']['applicable_trace_vehicle_speeds'] = applicable_trace_ml['Vehicle Speeds']
    # results[case_no]['matlab']['ts_trace_times'] = 
    results[case_no]['matlab']['ts_gears_out'] = gears_output_ml[0]
    results[case_no]['matlab']['ts_gears_correction_out'] = gears_corrections_output_ml[0]
    results[case_no]['matlab']['ts_required_vehicle_speeds_out'] = required_vehicles_speed_output_ml[0]
    results[case_no]['matlab']['ts_required_powers_out'] = required_powers_output_ml[0]
    results[case_no]['matlab']['ts_clutch_disengaged_out'] = clutch_disengaged_output_ml[0]
    results[case_no]['matlab']['ts_clutch_undefined_out'] = clutch_undefined_output_ml[0]
    results[case_no]['matlab']['ts_clutch_hst_out'] = clutch_HST_output_ml[0]

Wall time: 1min 9s


## Create hashes and sums from Matlab and Python results

In [7]:
results_sums = dict()
results_hashes = dict()

variables = list(results[1]['python'].keys())

for case_no in range(1, 126): 
    results_sums[case_no] = dict(); results_hashes[case_no] = dict()
    results_sums[case_no]['python'] = dict(); results_hashes[case_no]['python'] = dict()
    results_sums[case_no]['matlab'] = dict(); results_hashes[case_no]['matlab'] = dict()
    for var in variables: 
        if var in list(results[case_no]['python'].keys()) and var in list(results[case_no]['matlab'].keys()):
            results_hashes[case_no]['matlab'][var] = results[case_no]['matlab'][var]
            results_hashes[case_no]['python'][var] = results[case_no]['python'][var]
                
            results_hashes[case_no]['matlab'][var] = \
                            hash_pandas_object(results_hashes[case_no]['matlab'][var]).sum()
            results_hashes[case_no]['python'][var] = \
                            hash_pandas_object(results_hashes[case_no]['python'][var]).sum()
            
            if var == 'summary': 
                for col in list(column_rename.values()): 
                    results_hashes[case_no]['matlab'][var + '-' + col] = \
                                    hash_pandas_object(results[case_no]['matlab'][var][col]).sum()
                    results_hashes[case_no]['python'][var + '-' + col] = \
                                    hash_pandas_object(results[case_no]['python'][var][col]).sum()
                
        
            if var not in ['summary']:
                if (results[case_no]['python'][var].dtypes == 'float64') or \
                                            (results[case_no]['python'][var].dtypes == 'int64'):

                    results_sums[case_no]['matlab'][var] = results[case_no]['matlab'][var].sum()
                    results_sums[case_no]['python'][var] = results[case_no]['python'][var].sum()
            
            if var in ['summary']:
                for col in list(column_rename.values()): 
                    results_sums[case_no]['matlab'][var + '-' + col] = \
                                    results[case_no]['matlab'][var][col].sum()
                    results_sums[case_no]['python'][var + '-' + col] = \
                                    results[case_no]['python'][var][col].sum()

### Hashes

In [8]:
create_df_comparison_from_dict_results(results_hashes, bool_comparison=True)

### Sums (when they apply)

In [9]:
create_df_comparison_from_dict_results(results_sums, bool_comparison=False)

## Different columns checked manually

In this section we are going to check manually the results that are not True in the *Hashes* section.

* Summary:
    * **Required To Rated Power Ratio:**
    
      As you can see in the Required To Rated Power Ratio column in the *Sums (when they apply)* table/matrix the difference between Python and MATLAB results are different in the 4th decimal, for example:

In [10]:
cases = [7, 12, 43, 70, 85, 100, 123]
signal = 'Required To Rated Power Ratio'

In [11]:
for case in cases:
    print('The python result is %f and the matlab result is %f'%(results[case]['python']['summary'][signal], results[case]['matlab']['summary'][signal]))

The python result is 0.888476 and the matlab result is 0.888000
The python result is 0.426175 and the matlab result is 0.426000
The python result is 1.282098 and the matlab result is 1.282000
The python result is 0.247907 and the matlab result is 0.248000
The python result is 0.737843 and the matlab result is 0.738000
The python result is 1.198487 and the matlab result is 1.198000
The python result is 1.197958 and the matlab result is 1.198000


As you can see the difference is always in the 4th decimal. The reason is that in the MATLAB code the values are rounded to the 4th decimal, while in Python there is not rounding at all. If we round the values in Python, we obtain the same results:

In [12]:
for case in range(1, 126):
    print('Case %i: The python result is %.10f and the matlab result is %.10f, The difference is: %.10f'%(case, round(results[case]['python']['summary'][signal], 3), results[case]['matlab']['summary'][signal], round(results[case]['python']['summary'][signal], 3) - results[case]['matlab']['summary'][signal]))

Case 1: The python result is 0.4280000000 and the matlab result is 0.4280000000, The difference is: 0.0000000000
Case 2: The python result is 0.3610000000 and the matlab result is 0.3610000000, The difference is: 0.0000000000
Case 3: The python result is 0.3700000000 and the matlab result is 0.3700000000, The difference is: 0.0000000000
Case 4: The python result is 0.4720000000 and the matlab result is 0.4720000000, The difference is: -0.0000000000
Case 5: The python result is 0.5460000000 and the matlab result is 0.5460000000, The difference is: 0.0000000000
Case 6: The python result is 0.4880000000 and the matlab result is 0.4880000000, The difference is: 0.0000000000
Case 7: The python result is 0.8880000000 and the matlab result is 0.8880000000, The difference is: 0.0000000000
Case 8: The python result is 0.7490000000 and the matlab result is 0.7490000000, The difference is: -0.0000000000
Case 9: The python result is 0.7990000000 and the matlab result is 0.7990000000, The differenc

As you can see the values are equal. In this case, we can consider that the results are correct, as the discrepancy occurs due to the number of decimals that we use in Python. 

   * **Total Distance:**
   
       Although in some cases we have different values, in the Sums (when they apply) section the difference between Python and MATLAB is equal to 0, as the values are equal. We check these cases manually: 

In [13]:
cases = [25, 51, 57, 58, 77, 81, 106, 108, 119, 121]
signal = 'Total Distance'

In [14]:
for case in cases:
    print('The python result is %.10f and the matlab result is %.10f'%(results[case]['python']['summary'][signal], results[case]['matlab']['summary'][signal]))

The python result is 22546.0000000000 and the matlab result is 22546.0000000000
The python result is 22546.0000000000 and the matlab result is 22546.0000000000
The python result is 22546.0000000000 and the matlab result is 22546.0000000000
The python result is 23216.0000000000 and the matlab result is 23216.0000000000
The python result is 23216.0000000000 and the matlab result is 23216.0000000000
The python result is 22546.0000000000 and the matlab result is 22546.0000000000
The python result is 22546.0000000000 and the matlab result is 22546.0000000000
The python result is 22546.0000000000 and the matlab result is 22546.0000000000
The python result is 22445.0000000000 and the matlab result is 22445.0000000000
The python result is 22966.0000000000 and the matlab result is 22966.0000000000


The difference is:

In [15]:
for case in range(1, 126):
    print('The python result is %.10f and the matlab result is %.10f. The difference is: %.10f'%(results[case]['python']['summary'][signal], results[case]['matlab']['summary'][signal], results[case]['python']['summary'][signal] - results[case]['matlab']['summary'][signal]))

The python result is 23266.3000000000 and the matlab result is 23266.3000000000. The difference is: 0.0000000000
The python result is 23266.3000000000 and the matlab result is 23266.3000000000. The difference is: 0.0000000000
The python result is 23266.3000000000 and the matlab result is 23266.3000000000. The difference is: 0.0000000000
The python result is 23266.3000000000 and the matlab result is 23266.3000000000. The difference is: 0.0000000000
The python result is 23266.3000000000 and the matlab result is 23266.3000000000. The difference is: 0.0000000000
The python result is 23266.3000000000 and the matlab result is 23266.3000000000. The difference is: 0.0000000000
The python result is 23228.7000000000 and the matlab result is 23228.7000000000. The difference is: 0.0000000000
The python result is 23266.3000000000 and the matlab result is 23266.3000000000. The difference is: 0.0000000000
The python result is 23266.3000000000 and the matlab result is 23266.3000000000. The difference 

As you can see, if we round the python results, the difference is equal to 0.
   
   * **Adjusted Max95 Engine Speed:**
   
       In many cases the values are different, so we can see in the Sums (when they apply) section the difference is not equal to 0. This happens because in MATLAB we always round values to the second decimal:

In [16]:
cases = [25, 51, 57, 58, 77, 81, 106, 108, 119, 121]
signal = 'Adjusted Max95 Engine Speed'

In [17]:
for case in cases:
    print('The python result is %.10f and the matlab result is %.10f'%(results[case]['python']['summary'][signal], results[case]['matlab']['summary'][signal]))

The python result is 4141.8439716312 and the matlab result is 4141.8400000000
The python result is 4141.8439716312 and the matlab result is 4141.8400000000
The python result is 4141.8439716312 and the matlab result is 4141.8400000000
The python result is 3793.1818181818 and the matlab result is 3793.1800000000
The python result is 4141.7419884963 and the matlab result is 4141.7400000000
The python result is 3301.8485915493 and the matlab result is 3301.8500000000
The python result is 4141.8439716312 and the matlab result is 4141.8400000000
The python result is 4141.8439716312 and the matlab result is 4141.8400000000
The python result is 5462.3616236162 and the matlab result is 5462.3600000000
The python result is 3619.5899772210 and the matlab result is 3619.5900000000


If we round the results of Python to the second decimal, we can see that the values are equal:

In [18]:
for case in range(1, 126):
    print('Case %i: The python result is %.10f and the matlab result is %.10f. The difference is: %.10f'%(case, round(results[case]['python']['summary'][signal], 2), results[case]['matlab']['summary'][signal], round(results[case]['python']['summary'][signal], 2) - results[case]['matlab']['summary'][signal]))

Case 1: The python result is 4379.7500000000 and the matlab result is 4379.7500000000. The difference is: 0.0000000000
Case 2: The python result is 6913.0100000000 and the matlab result is 6913.0100000000. The difference is: 0.0000000000
Case 3: The python result is 6685.4800000000 and the matlab result is 6685.4800000000. The difference is: 0.0000000000
Case 4: The python result is 4387.9000000000 and the matlab result is 4387.9000000000. The difference is: 0.0000000000
Case 5: The python result is 6076.6700000000 and the matlab result is 6076.6700000000. The difference is: 0.0000000000
Case 6: The python result is 6374.1100000000 and the matlab result is 6374.1100000000. The difference is: 0.0000000000
Case 7: The python result is 3602.5700000000 and the matlab result is 3602.5700000000. The difference is: 0.0000000000
Case 8: The python result is 3230.0000000000 and the matlab result is 3230.0000000000. The difference is: 0.0000000000
Case 9: The python result is 7689.8300000000 and

As you can see, if we round, we obtain exactly the same results and the difference is equal to 0. The differences occurs due to the round method in the MATLAB version of the tool.

   * **Max Engine Speed Cycle Output:**
   
       In many cases the values are different, but in the Sums (when they apply) section the difference between Python and MATLAB is visible after the 3rd decimal point:

In [19]:
cases = [25, 51, 57, 58, 77, 81, 106, 108, 119, 121]
signal = 'Max Engine Speed Cycle Output'

In [20]:
for case in cases:
    print('The python result is %f and the matlab result is %f'%(results[case]['python']['summary'][signal], results[case]['matlab']['summary'][signal]))

The python result is 3293.316000 and the matlab result is 3293.320000
The python result is 2683.980000 and the matlab result is 2683.980000
The python result is 2683.980000 and the matlab result is 2683.980000
The python result is 3063.606000 and the matlab result is 3063.610000
The python result is 4036.200000 and the matlab result is 4036.200000
The python result is 2659.800000 and the matlab result is 2659.800000
The python result is 2683.980000 and the matlab result is 2683.980000
The python result is 2683.980000 and the matlab result is 2683.980000
The python result is 3440.000000 and the matlab result is 3440.000000
The python result is 2667.500000 and the matlab result is 2667.500000


As you can see the difference is always in the 3rd decimal. The reason is that in the MATLAB code the values are rounded to the 2nd decimal, while in Python there is not rounding at all. If we round the values in Python, we obtain the same results:

In [21]:
for case in range(1, 126):
    print('Case %i: The python result is %.10f and the matlab result is %.10f. The difference is: %.10f'%(case, round(results[case]['python']['summary'][signal], 2), results[case]['matlab']['summary'][signal], round(results[case]['python']['summary'][signal], 2) - results[case]['matlab']['summary'][signal]))

Case 1: The python result is 2356.8400000000 and the matlab result is 2356.8400000000. The difference is: 0.0000000000
Case 2: The python result is 2882.0400000000 and the matlab result is 2882.0400000000. The difference is: 0.0000000000
Case 3: The python result is 3222.1000000000 and the matlab result is 3222.1000000000. The difference is: 0.0000000000
Case 4: The python result is 2656.2000000000 and the matlab result is 2656.2000000000. The difference is: 0.0000000000
Case 5: The python result is 4167.4600000000 and the matlab result is 4167.4600000000. The difference is: 0.0000000000
Case 6: The python result is 4398.5500000000 and the matlab result is 4398.5500000000. The difference is: 0.0000000000
Case 7: The python result is 2781.4300000000 and the matlab result is 2781.4300000000. The difference is: 0.0000000000
Case 8: The python result is 4130.7000000000 and the matlab result is 4130.7000000000. The difference is: 0.0000000000
Case 9: The python result is 6118.5800000000 and

In this case after rounding, you can see that the cases 55 and 122 have a difference of 0.01 between python and matlab results, but this difference is acceptable, therefore we can say the outputs obtained with python tool are correct.

In [22]:
for case in [55, 122]:
    print('Case %i: The python result is %.10f and the matlab result is %.10f. The difference is: %.10f'%(case, results[case]['python']['summary'][signal], results[case]['matlab']['summary'][signal], results[case]['python']['summary'][signal] - results[case]['matlab']['summary'][signal]))

Case 55: The python result is 2580.0450000000 and the matlab result is 2580.0500000000. The difference is: -0.0050000000
Case 122: The python result is 3184.0250000000 and the matlab result is 3184.0300000000. The difference is: -0.0050000000


In this case the difference is the rounded method used by default in Python, i.e. Python rounds to the nearest even value and MATLAB rounds to the nearest integer, but if we use the round half up method in Python:

In [23]:
def _round_half_up(n, decimals=0):
    import math
    multiplier = 10 ** decimals
    return math.floor(n*multiplier + 0.5) / multiplier

for case in [55, 122]:
    print('Case %i: The python result is %.10f and the matlab result is %.10f. The difference is: %.10f'%(case, _round_half_up(results[case]['python']['summary'][signal] * 100)/100, results[case]['matlab']['summary'][signal], _round_half_up(results[case]['python']['summary'][signal]*100)/100 - results[case]['matlab']['summary'][signal]))

Case 55: The python result is 2580.0500000000 and the matlab result is 2580.0500000000. The difference is: 0.0000000000
Case 122: The python result is 3184.0300000000 and the matlab result is 3184.0300000000. The difference is: 0.0000000000


As you can see, if we use in Python the same method to round values that is used in MATLAB we can obtain exactly the same values in these cases. 

   * **Max Engine Speed Reachable Output:**
   
       In many cases the values are different, but in the Sums (when they apply) section the difference between Python and MATLAB is visible after the 3rd decimal point:

In [24]:
cases = [2, 5, 9, 15, 26, 44, 59, 67, 73, 92, 119]
signal = 'Max Engine Speed Reachable Output'

In [25]:
for case in cases:
    print('The python result is %f and the matlab result is %f'%(results[case]['python']['summary'][signal], results[case]['matlab']['summary'][signal]))

The python result is 4980.455000 and the matlab result is 4980.450000
The python result is 5579.892000 and the matlab result is 5579.890000
The python result is 6705.740000 and the matlab result is 6705.740000
The python result is 5405.400000 and the matlab result is 5405.400000
The python result is 3972.672000 and the matlab result is 3972.670000
The python result is 4028.874000 and the matlab result is 4028.870000
The python result is 3174.197000 and the matlab result is 3174.200000
The python result is 6751.200000 and the matlab result is 6751.200000
The python result is 8064.000000 and the matlab result is 8064.000000
The python result is 4839.280000 and the matlab result is 4839.280000
The python result is 5559.900000 and the matlab result is 5559.900000


As you can see the difference is always in the 3rd decimal. The reason is that in the MATLAB code the values are rounded to the 2nd decimal, while in Python there is not rounding at all. If we round the values in Python, we obtain the same results:

In [26]:
for case in range(1, 126):
    print('Case %i: The python result is %.10f and the matlab result is %.10f. The difference is: %.10f'%(case, round(results[case]['python']['summary'][signal], 2), results[case]['matlab']['summary'][signal], round(results[case]['python']['summary'][signal], 2) - results[case]['matlab']['summary'][signal]))

Case 1: The python result is 3773.0900000000 and the matlab result is 3773.0900000000. The difference is: 0.0000000000
Case 2: The python result is 4980.4600000000 and the matlab result is 4980.4500000000. The difference is: 0.0100000000
Case 3: The python result is 5173.0300000000 and the matlab result is 5173.0300000000. The difference is: 0.0000000000
Case 4: The python result is 4009.5900000000 and the matlab result is 4009.5900000000. The difference is: 0.0000000000
Case 5: The python result is 5579.8900000000 and the matlab result is 5579.8900000000. The difference is: 0.0000000000
Case 6: The python result is 6060.1500000000 and the matlab result is 6060.1500000000. The difference is: 0.0000000000
Case 7: The python result is 3071.5200000000 and the matlab result is 3071.5200000000. The difference is: 0.0000000000
Case 8: The python result is 4486.2000000000 and the matlab result is 4486.2000000000. The difference is: 0.0000000000
Case 9: The python result is 6705.7400000000 and

We notice that case 2 has a difference of 0.01 between Python and MATLAB results. The rounded value of 4980.455000 is 4980.460000 and the MATLAB value is 4980.450000, The difference is 0.005 and therefore it is acceptable.

   * **Max Engine Speed Output:**
   
       In some cases the values are different, but in the Sums (when they apply) section the difference between Python and MATLAB is visible after the 3rd decimal point:

In [27]:
cases = [25, 51, 57, 58, 77, 81, 106, 108, 119, 121]
signal = 'Max Engine Speed Output'

In [28]:
for case in cases:
    print('The python result is %.10f and the matlab result is %.10f'%(results[case]['python']['summary'][signal], results[case]['matlab']['summary'][signal]))

The python result is 4141.8439716312 and the matlab result is 4141.8400000000
The python result is 4141.8439716312 and the matlab result is 4141.8400000000
The python result is 4141.8439716312 and the matlab result is 4141.8400000000
The python result is 3793.1818181818 and the matlab result is 3793.1800000000
The python result is 4563.2000000000 and the matlab result is 4563.2000000000
The python result is 3301.8485915493 and the matlab result is 3301.8500000000
The python result is 4141.8439716312 and the matlab result is 4141.8400000000
The python result is 4141.8439716312 and the matlab result is 4141.8400000000
The python result is 5559.9000000000 and the matlab result is 5559.9000000000
The python result is 3619.5899772210 and the matlab result is 3619.5900000000


As you can see the difference is always in the 3rd decimal. The reason is that in the MATLAB code the values are rounded to the 2nd decimal, while in Python there is not rounding at all. If we round the values in Python, we obtain the same results:

In [29]:
for case in range(1, 126):
    print('Case %i: The python result is %.10f and the matlab result is %.10f. The difference is: %.10f'%(case, round(results[case]['python']['summary'][signal], 2), results[case]['matlab']['summary'][signal], round(results[case]['python']['summary'][signal], 2) - results[case]['matlab']['summary'][signal]))

Case 1: The python result is 4379.7500000000 and the matlab result is 4379.7500000000. The difference is: 0.0000000000
Case 2: The python result is 6913.0100000000 and the matlab result is 6913.0100000000. The difference is: 0.0000000000
Case 3: The python result is 6685.4800000000 and the matlab result is 6685.4800000000. The difference is: 0.0000000000
Case 4: The python result is 4387.9000000000 and the matlab result is 4387.9000000000. The difference is: 0.0000000000
Case 5: The python result is 6076.6700000000 and the matlab result is 6076.6700000000. The difference is: 0.0000000000
Case 6: The python result is 6374.1100000000 and the matlab result is 6374.1100000000. The difference is: 0.0000000000
Case 7: The python result is 3602.5700000000 and the matlab result is 3602.5700000000. The difference is: 0.0000000000
Case 8: The python result is 4486.2000000000 and the matlab result is 4486.2000000000. The difference is: 0.0000000000
Case 9: The python result is 7689.8300000000 and

If we round the results of Python to the second decimal, we can see that the values are equal.

   * **Max Vehicle Speed Cycle Output:**
   
       In some cases the values are different, but in the Sums (when they apply) section the difference between Python and MATLAB is visible after the 1st decimal point:

In [30]:
cases = [26, 33, 35, 41, 75, 76, 79, 80, 87, 101, 114, 117, 118, 119, 120, 121]
signal = 'Max Vehicle Speed Cycle Output'

In [31]:
for case in cases:
    print('The python result is %f and the matlab result is %f'%(results[case]['python']['summary'][signal], results[case]['matlab']['summary'][signal]))

The python result is 61.942082 and the matlab result is 61.900000
The python result is 108.209965 and the matlab result is 108.200000
The python result is 116.583256 and the matlab result is 116.600000
The python result is 63.020478 and the matlab result is 63.000000
The python result is 111.301074 and the matlab result is 111.300000
The python result is 124.211664 and the matlab result is 124.200000
The python result is 57.762261 and the matlab result is 57.800000
The python result is 121.439735 and the matlab result is 121.400000
The python result is 117.000000 and the matlab result is 117.000000
The python result is 108.209965 and the matlab result is 108.200000
The python result is 108.209965 and the matlab result is 108.200000
The python result is 55.000000 and the matlab result is 55.000000
The python result is 55.000000 and the matlab result is 55.000000
The python result is 80.000000 and the matlab result is 80.000000
The python result is 100.000000 and the matlab result is 100

As you can see the difference is always in the 1st decimal. The reason is that in the MATLAB code the values are rounded to the 1st decimal, while in Python there is not rounding at all. If we round the values in Python, we obtain the same results:

In [32]:
for case in cases:
    print('Case %i: The python result is %.10f and the matlab result is %.10f. The difference is: %.10f'%(case, round(results[case]['python']['summary'][signal], 1), results[case]['matlab']['summary'][signal], round(results[case]['python']['summary'][signal], 1) - results[case]['matlab']['summary'][signal]))

Case 26: The python result is 61.9000000000 and the matlab result is 61.9000000000. The difference is: 0.0000000000
Case 33: The python result is 108.2000000000 and the matlab result is 108.2000000000. The difference is: 0.0000000000
Case 35: The python result is 116.6000000000 and the matlab result is 116.6000000000. The difference is: 0.0000000000
Case 41: The python result is 63.0000000000 and the matlab result is 63.0000000000. The difference is: 0.0000000000
Case 75: The python result is 111.3000000000 and the matlab result is 111.3000000000. The difference is: 0.0000000000
Case 76: The python result is 124.2000000000 and the matlab result is 124.2000000000. The difference is: 0.0000000000
Case 79: The python result is 57.8000000000 and the matlab result is 57.8000000000. The difference is: 0.0000000000
Case 80: The python result is 121.4000000000 and the matlab result is 121.4000000000. The difference is: 0.0000000000
Case 87: The python result is 117.0000000000 and the matlab re

As we can see, if we round the python results, the difference is equal to 0.

   * **Max Vehicle Speed Reachable Output:**
   
       In some cases the values are different, but in the Sums (when they apply) section the difference between Python and MATLAB is visible after the 1st decimal point:

In [33]:
cases = [7, 26, 41, 56, 67, 68, 73, 74, 78, 81, 84, 88, 118]
signal = 'Max Vehicle Speed Reachable Output'

In [34]:
for case in cases:
    print('The python result is %f and the matlab result is %f'%(results[case]['python']['summary'][signal], results[case]['matlab']['summary'][signal]))

The python result is 144.000000 and the matlab result is 144.000000
The python result is 68.400000 and the matlab result is 68.400000
The python result is 64.000000 and the matlab result is 64.000000
The python result is 144.000000 and the matlab result is 144.000000
The python result is 348.000000 and the matlab result is 348.000000
The python result is 312.300000 and the matlab result is 312.300000
The python result is 252.000000 and the matlab result is 252.000000
The python result is 145.000000 and the matlab result is 145.000000
The python result is 64.400000 and the matlab result is 64.400000
The python result is 147.000000 and the matlab result is 147.000000
The python result is 228.000000 and the matlab result is 228.000000
The python result is 133.000000 and the matlab result is 133.000000
The python result is 68.400000 and the matlab result is 68.400000


In [35]:
for case in cases:
    print('Case %i: The python result is %.10f and the matlab result is %.10f. The difference is: %.10f'%(case, round(results[case]['python']['summary'][signal], 1), results[case]['matlab']['summary'][signal], round(results[case]['python']['summary'][signal], 1) - results[case]['matlab']['summary'][signal]))

Case 7: The python result is 144.0000000000 and the matlab result is 144.0000000000. The difference is: 0.0000000000
Case 26: The python result is 68.4000000000 and the matlab result is 68.4000000000. The difference is: 0.0000000000
Case 41: The python result is 64.0000000000 and the matlab result is 64.0000000000. The difference is: 0.0000000000
Case 56: The python result is 144.0000000000 and the matlab result is 144.0000000000. The difference is: 0.0000000000
Case 67: The python result is 348.0000000000 and the matlab result is 348.0000000000. The difference is: 0.0000000000
Case 68: The python result is 312.3000000000 and the matlab result is 312.3000000000. The difference is: 0.0000000000
Case 73: The python result is 252.0000000000 and the matlab result is 252.0000000000. The difference is: 0.0000000000
Case 74: The python result is 145.0000000000 and the matlab result is 145.0000000000. The difference is: 0.0000000000
Case 78: The python result is 64.4000000000 and the matlab re

As we can see, if we round the python results, the difference is equal to 0.

   * **Checksum Vx Gear Output:**
   
       In some cases the values are different, but in the Sums (when they apply) section the difference between Python and MATLAB is visible after the 1st decimal point: 

In [36]:
cases = [1, 10, 12, 43, 47, 68, 70, 89, 92, 96, 109]
signal = 'Checksum Vx Gear Output'

In [37]:
for case in cases:
    print('The python result is %.10f and the matlab result is %.10f'%(results[case]['python']['summary'][signal], results[case]['matlab']['summary'][signal]))

The python result is 404441.0000000000 and the matlab result is 404441.0000000000
The python result is 362252.0000000000 and the matlab result is 362252.0000000000
The python result is 369438.0000000000 and the matlab result is 369438.0000000000
The python result is 456116.0000000000 and the matlab result is 456116.0000000000
The python result is 413649.0000000000 and the matlab result is 413649.0000000000
The python result is 399611.0000000000 and the matlab result is 399611.0000000000
The python result is 434233.0000000000 and the matlab result is 434233.0000000000
The python result is 389721.0000000000 and the matlab result is 389721.0000000000
The python result is 417064.0000000000 and the matlab result is 417064.0000000000
The python result is 430737.0000000000 and the matlab result is 430737.0000000000
The python result is 330891.0000000000 and the matlab result is 330891.0000000000


In [38]:
for case in cases:
    print('Case %i: The python result is %.10f and the matlab result is %.10f. The difference is: %.10f'%(case, results[case]['python']['summary'][signal], results[case]['matlab']['summary'][signal], results[case]['python']['summary'][signal] - results[case]['matlab']['summary'][signal]))

Case 1: The python result is 404441.0000000000 and the matlab result is 404441.0000000000. The difference is: 0.0000000000
Case 10: The python result is 362252.0000000000 and the matlab result is 362252.0000000000. The difference is: 0.0000000000
Case 12: The python result is 369438.0000000000 and the matlab result is 369438.0000000000. The difference is: 0.0000000000
Case 43: The python result is 456116.0000000000 and the matlab result is 456116.0000000000. The difference is: 0.0000000000
Case 47: The python result is 413649.0000000000 and the matlab result is 413649.0000000000. The difference is: 0.0000000000
Case 68: The python result is 399611.0000000000 and the matlab result is 399611.0000000000. The difference is: 0.0000000000
Case 70: The python result is 434233.0000000000 and the matlab result is 434233.0000000000. The difference is: 0.0000000000
Case 89: The python result is 389721.0000000000 and the matlab result is 389721.0000000000. The difference is: 0.0000000000
Case 92: 

As we can see, if we round the python results, the difference is equal to 0.



* **original_trace_vehicle_speeds:**

    In this case ,in the Sums (when they apply) section, the difference between Python and MATLAB values is equal to 0. However, the comparison produces a negative result as the number of decimals stored by Python and MATLAB is different: 

In [39]:
case = 17
signal = 'original_trace_vehicle_speeds'

In [40]:
print(results[case]['python'][signal].loc[results[case]['python'][signal] != results[case]['matlab'][signal]])

385    0.9
Name: Vehicle Speeds, dtype: float64


In [41]:
print(results[case]['matlab'][signal].loc[results[case]['matlab'][signal] != results[case]['python'][signal]])

385    0.9
Name: Vehicle Speeds, dtype: float64


If we print the results without rounding, we can see that the values are not equal:

In [42]:
print(results[case]['matlab'][signal][385])

0.9


In [43]:
print(results[case]['python'][signal][385])

0.8999999999999999


We notice that the value is the same, but in Python it is not rounded to the 1st decimal: 

In [44]:
results[case]['matlab'][signal][385] == round(results[case]['python'][signal][385], 1)

True

This example is applicable to all cases.

* **applicable_trace_vehicle_speeds:**

    In this case ,in the Sums (when they apply) section, the difference between Python and MATLAB values is equal to 0. However, the comparison produces a negative result as the number of decimals stored for float numbers by Python and MATLAB is different: 

In [45]:
case = 61
signal = 'applicable_trace_vehicle_speeds'

In [46]:
print(results[case]['python'][signal].loc[results[case]['python'][signal] != results[case]['matlab'][signal]])

385    0.9
Name: Vehicle Speeds, dtype: float64


In [47]:
print(results[case]['matlab'][signal].loc[results[case]['matlab'][signal] != results[case]['python'][signal]])

385    0.9
Name: Vehicle Speeds, dtype: float64


In [48]:
print(results[case]['matlab'][signal][385])

0.9


In [49]:
print(results[case]['python'][signal][385])

0.8999999999999999


We notice that the value is the same, but in Python it is not rounded to the 1st decimal: 

In [50]:
results[case]['matlab'][signal][385] == round(results[case]['python'][signal][385], 1)

True

This example is applicable to all cases.

* **ts_required_vehicle_speeds_out:**

    In this case ,in the Sums (when they apply) section, the difference between Python and MATLAB values is equal to 0. However, the comparison produces a negative result as the number of decimals stored for float numbers by Python and MATLAB is different:

In [51]:
case = 43
signal = 'ts_required_vehicle_speeds_out'

In [52]:
for case in [43, 44, 120]:
    print("The case %i"%(case))
    print(results[case]['python'][signal].loc[results[case]['python'][signal] != results[case]['matlab'][signal]])
    print(results[case]['matlab'][signal].loc[results[case]['matlab'][signal] != results[case]['python'][signal]])

The case 43
1427     39.122138
1567    102.659159
1568    103.925445
1569    105.105562
1570    106.197986
1571    107.208593
1572    108.143825
1573    109.009555
1718    114.898690
1719    115.268340
1720    115.611087
1721    115.928908
1722    116.223630
1723    116.496946
Name: Required Vehicle Speeds Output, dtype: float64
1427     39.122138
1567    102.659159
1568    103.925445
1569    105.105562
1570    106.197986
1571    107.208593
1572    108.143825
1573    109.009555
1718    114.898690
1719    115.268340
1720    115.611087
1721    115.928908
1722    116.223630
1723    116.496946
Name: 0, dtype: float64
The case 44
539      18.272278
1427     39.122138
1567    102.659159
1568    103.925445
1569    105.105562
1570    106.197986
1571    107.208593
1572    108.143825
1573    109.009555
1718    114.898690
1719    115.268340
1720    115.611087
1721    115.928908
1722    116.223630
1723    116.496946
Name: Required Vehicle Speeds Output, dtype: float64
539      18.272278
1427     3

In [53]:
print(results[case]['matlab'][signal][1427])

39.122138005542


In [54]:
print(results[case]['python'][signal][1427])

39.12213800554203


In [55]:
results[case]['matlab'][signal][1546] == round(results[case]['python'][signal][1546], 12)

True

If we apply the same comparison for the different results of the various cases,  we can see that the result is the same. This is due to the number of decimals stored for float numbers by Python and MATLAB. If we round the python results to the same number of decimals of the values in MATLAB, we can see that the result is exactly the same. 

* **ts_required_powers_out:**

    In this case ,in the Sums (when they apply) section, the difference between Python and MATLAB values is equal to 0. However, the comparison produces a negative result as the number of decimals stored for float numbers by Python and MATLAB is different:

In [56]:
case = 1
signal = 'ts_required_powers_out'

In [57]:
print(results[case]['python'][signal].loc[results[case]['python'][signal] != results[case]['matlab'][signal]])

12       0.051647
13       0.944599
15       4.848376
16       7.490123
17      11.969524
          ...    
1789    -5.114018
1791    -5.783788
1792    -5.268000
1793    -3.617494
1794    -3.360553
Name: Required Powers Output, Length: 1422, dtype: float64


In [58]:
print(results[case]['matlab'][signal].loc[results[case]['matlab'][signal] != results[case]['python'][signal]])

12       0.051647
13       0.944599
15       4.848376
16       7.490123
17      11.969524
          ...    
1789    -5.114018
1791    -5.783788
1792    -5.268000
1793    -3.617494
1794    -3.360553
Name: 0, Length: 1422, dtype: float64


In [59]:
print(results[case]['matlab'][signal][12])

0.0516474785185185


In [60]:
print(results[case]['python'][signal][12])

0.05164747851851852


In [61]:
results[case]['matlab'][signal][12] == round(results[case]['python'][signal][12], 16)

True

We apply the same comparison for the different results of the various cases,  we can see that the result is the same. This is due to the number of decimals stored for float numbers by Python and MATLAB. If we round the python results to the same number of decimals of the values in MATLAB, we can see that the result is exactly the same. 

# Conclusion:

## After applying the hash test for all values, we can notice that some values are different (see section Hashes). The difference found in the hash is due to the decimals stored in each execution, since Python stores a different number of decimals for float numbers than MATLAB. Finally, if we compare the results rounding the Python results to the same decimals stored by MATLAB, we can conclude that the results are correct. 